In [1]:
from keras.datasets import fashion_mnist,mnist
import wandb
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import copy
import argparse
from types import SimpleNamespace


from utilities.NeuralNetwork import NN
from utilities.HelperFunctions import OneHotEncoder
from utilities.config import * # reading global variables 


def pre_process(x):
    '''
    reshape and normalized the data to bring to 0-1 scale.
    '''
    x=x.reshape(-1,784)
    x=x/255
    return x

def load_data(dataset=fmnist_dataset,split_size=valid_split_size):
    
    '''
    loads and returns data after doing train-valid split.
    '''
    if dataset==fmnist_dataset:
        (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    elif dataset==mnist_dataset:
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    
    x_train,x_valid=x_train[:int(len(x_train)*split_size)],x_train[int(len(x_train)*split_size):] #splitting train into train and valid
    y_train,y_valid=y_train[:int(len(y_train)*split_size)],y_train[int(len(y_train)*split_size):]

    x_train=pre_process(x_train)
    x_valid=pre_process(x_valid)
    x_test=pre_process(x_test) 
    
    one_hot=OneHotEncoder(10)
    y_train=one_hot.transform(y_train)
    y_valid=one_hot.transform(y_valid)
    y_test=one_hot.transform(y_test)
    
    
    return x_train,y_train,x_valid,y_valid,x_test,y_test

x_train,y_train,x_valid,y_valid,x_test,y_test=load_data(fmnist_dataset,1-valid_split_size)
    

In [2]:
sweep_config = {
    'method': 'grid', #grid, random,#bayes
    'name' : 'CrossEntropy_VS_MSE',
    'metric': {
      'name': 'valid accuracy',
      'goal': 'maximize'  
    },
    'parameters': {
        'epochs': {
            'values': [10]
        },
        'num_layers':{
            'values':[3]
        },
         'hidden_size':{
            'values':[64]
        },
        'weight_decay':{
            'values':[0.0005]
        },
         'learning_rate':{
            'values':[0.0001]
        },
         'optimizer': {
            'values': ['nadam']           
        },
        'batch_size': {
            'values': [16]
        },
         'weight_init':{
            'values':['xavier']            
        },
        'activation': {
            'values': ['tanh']
        }, 
        'loss': {
            'values': [entropy_loss,squared_loss]
        }, 
        
        
    }
}

sweep_id = wandb.sweep(sweep=sweep_config, project='temp')

Create sweep with ID: 43rvobw4
Sweep URL: https://wandb.ai/cs22m080/temp/sweeps/43rvobw4


In [3]:
def main():
    with wandb.init() as run:
        params={}
        params=dict(wandb.config)
        params=SimpleNamespace(**params)
        
        layers=[params.hidden_size]*params.num_layers
        run_name="loss_"+wandb.config.loss+"-opt_"+wandb.config.optimizer+"-ac_"+wandb.config.activation+"-batch_"+str(wandb.config.batch_size)\
                + "-nlayers_"+str(wandb.config.num_layers)+"-lr_"+str(wandb.config.learning_rate)
        wandb.run.name=run_name
        layers.append(10)
        obj=NN(784,layers,params)
        train_losses,tr_ac,valid_losses,val_ac=obj.train(x_train.T,y_train,x_valid.T,y_valid)
    
wandb.agent(sweep_id, function=main,count=2)
wandb.finish()

wandb: Agent Starting Run: ll2hvidb with config:
wandb: 	activation: tanh
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.0001
wandb: 	loss: cross_entropy
wandb: 	num_layers: 3
wandb: 	optimizer: nadam
wandb: 	weight_decay: 0.0005
wandb: 	weight_init: xavier
wandb: Currently logged in as: cs22m080. Use `wandb login --relogin` to force relogin


epoch 1 : train loss = 0.59 valid loss = 0.43 train accuracy = 84.81 valid accuracy = 84.33
epoch 2 : train loss = 0.40 valid loss = 0.39 train accuracy = 86.61 valid accuracy = 85.82
epoch 3 : train loss = 0.36 valid loss = 0.37 train accuracy = 87.66 valid accuracy = 86.47
epoch 4 : train loss = 0.34 valid loss = 0.35 train accuracy = 88.40 valid accuracy = 87.17
epoch 5 : train loss = 0.33 valid loss = 0.35 train accuracy = 88.96 valid accuracy = 87.48
epoch 6 : train loss = 0.31 valid loss = 0.34 train accuracy = 89.36 valid accuracy = 87.70
epoch 7 : train loss = 0.30 valid loss = 0.34 train accuracy = 89.73 valid accuracy = 87.93
epoch 8 : train loss = 0.29 valid loss = 0.33 train accuracy = 90.07 valid accuracy = 88.12
epoch 9 : train loss = 0.28 valid loss = 0.33 train accuracy = 90.35 valid accuracy = 88.38
epoch 10 : train loss = 0.27 valid loss = 0.32 train accuracy = 90.65 valid accuracy = 88.47


train accuracy,▁▃▄▅▆▆▇▇██
train loss,█▄▃▃▂▂▂▁▁▁
valid accuracy,▁▄▅▆▆▇▇▇██
valid loss,█▅▄▃▂▂▂▁▁▁
train accuracy,90.65
train loss,0.27293
valid accuracy,88.46667
valid loss,0.32497


wandb: Agent Starting Run: fp3zzkl8 with config:
wandb: 	activation: tanh
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.0001
wandb: 	loss: mean_squared_error
wandb: 	num_layers: 3
wandb: 	optimizer: nadam
wandb: 	weight_decay: 0.0005
wandb: 	weight_init: xavier


epoch 1 : train loss = 0.30 valid loss = 0.23 train accuracy = 85.04 valid accuracy = 84.12
epoch 2 : train loss = 0.21 valid loss = 0.21 train accuracy = 86.56 valid accuracy = 85.52
epoch 3 : train loss = 0.19 valid loss = 0.19 train accuracy = 87.64 valid accuracy = 86.20
epoch 4 : train loss = 0.18 valid loss = 0.19 train accuracy = 88.31 valid accuracy = 86.95
epoch 5 : train loss = 0.17 valid loss = 0.18 train accuracy = 88.80 valid accuracy = 87.28
epoch 6 : train loss = 0.17 valid loss = 0.18 train accuracy = 89.15 valid accuracy = 87.67
epoch 7 : train loss = 0.16 valid loss = 0.18 train accuracy = 89.50 valid accuracy = 87.77
epoch 8 : train loss = 0.16 valid loss = 0.18 train accuracy = 89.80 valid accuracy = 87.87
epoch 9 : train loss = 0.15 valid loss = 0.17 train accuracy = 90.10 valid accuracy = 87.98
epoch 10 : train loss = 0.15 valid loss = 0.17 train accuracy = 90.33 valid accuracy = 87.93


train accuracy,▁▃▄▅▆▆▇▇██
train loss,█▄▃▂▂▂▂▁▁▁
valid accuracy,▁▄▅▆▇▇████
valid loss,█▅▄▃▂▂▂▁▁▁
train accuracy,90.32778
train loss,0.15009
valid accuracy,87.93333
valid loss,0.17377
